# PandasSQLWindow Examples

Despite all the brilliant and user-friendly functionalities making Pandas the preferred data-manipulation framework in Python (especially when compared to PySpark), I have often received questions/requests regarding how to conveniently perform SQL Window Function-like operations in Pandas especially when working on datasets with many logically-partitioned groups.


As a result, I have written the PandasSQLWindow class as an attempt to unify some features common in SQL and PySpark using our beloved Pandas. This notebook walks through some of the functionalities of (the not so creatively named) PandasSQLWindow.
**Both rolling and expanding windows can used.**

Currently implemented Window functions:
- shift
- lag
- lead
- last (last known, non-null value)
- rank
- count

- min
- max
- mean
- median
- quantile
- sum
- var
- std


**Tip:** use `help(Window)` to find out all parameters and available methods

In [3]:
import pandas as pd
import numpy as np

from PandasSQLWindow import Window

In [4]:
help(Window)

Help on class Window in module PandasSQLWindow.core:

class Window(builtins.object)
 |  Window(data, partition_by, order_by, ascending=True, rows_rolling=None, time_rolling=None)
 |  
 |  SQL Window Functions in a unified, simple Pandas API.
 |  Follows the ... PARTITION BY ... ORDER BY ... format from SQL.
 |  Especially helpful for working with data with many logically-partitioned 'groups' 
 |  or for those more familiar with Window Functions from SQL or Apache Spark.
 |  Commonly requested functions:
 |  last() - finds the last previously known non-nan value 
 |           before the current row, within the same group
 |  lag() - find the preceding value 
 |          before the current row, within the same group
 |  lead() - finds the succeeding value
 |           after the current row, within the same group
 |  The current list only serves to demonstrate a few functionalities
 |  and is by no means exhaustive. Please feel free to reach out with
 |  any suggestions or requests.
 |  P

In [5]:
df = pd.DataFrame({'group':['a', 'b', 'b', 'c', 'c', 'c'],
                   'timestamp':[1, 2, 1, 3, 2, 1], 
                   'value': [1, 2, 3, 4, np.nan, 6]})

df

,group,timestamp,value
0,a,1,1.0
1,b,2,2.0
2,b,1,3.0
3,c,3,4.0
4,c,2,NaN
5,c,1,6.0


In [7]:
w = Window(data=df, 
           partition_by='group', # alternatively, use a list to partition by multiple columns
           order_by='timestamp', 
           rows_rolling=2)

# count, lag, lead, last have access to the entire logical partition
# i.e. count can be used to get a "row number" for each partition
df['count'] = w.count() 
df['lag'] = w.lag('value', periods=1)
df['lead'] = w.lead('value')
df['last_known'] = w.last('value')

# since rows_rolling is specified, sum() will be a rolling sum
df['rolling_sum'] = w.sum('value')
# expanding sum on the entire partition is also available if explicitly called
df['expanding_sum'] = w.expanding_sum('value') 

# you can also explicitly specify rolling_mean() instead of mean()
df['rolling_mean'] = w.rolling_mean('value')
# expanding mean on the entire partition is also available if explicitly called
df['expanding_mean'] = w.expanding_mean('value') 

column_order = [
    'group',
    'timestamp',
    'value',
    'count',
    'lag',
    'lead',
    'last_known',
    'rolling_sum',
    'expanding_sum',
    'rolling_mean',
    'expanding_mean'
]

# Just for ease of reading:
df[column_order].sort_values(['group', 'timestamp'])

,group,timestamp,value,count,lag,lead,last_known,rolling_sum,expanding_sum,rolling_mean,expanding_mean
0,a,1,1.0,1,NaN,NaN,NaN,1.0,1.0,1.0,1.0
2,b,1,3.0,1,NaN,2.0,NaN,3.0,3.0,3.0,3.0
1,b,2,2.0,2,3.0,NaN,3.0,5.0,5.0,2.5,2.5
5,c,1,6.0,1,NaN,NaN,NaN,6.0,6.0,6.0,6.0
4,c,2,NaN,2,6.0,4.0,6.0,6.0,6.0,6.0,6.0
3,c,3,4.0,3,NaN,NaN,6.0,4.0,10.0,4.0,5.0
